In [ ]:
import pandas as pd
import time
from riotwatcher import LolWatcher, RiotWatcher, ApiError
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from tensorflow.keras.optimizers import Adam
import os
import random
import requests
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as keras
from scipy import stats

In [ ]:
API_KEY = input("Wklej tutaj swój klucz Riot API i naciśnij Enter: ")

REGION = 'eun1'
ROUTING = 'europe'
QUEUE_ID = 420
COUNT_MATCHES_PER_PLAYER = 20
TARGET_MATCHES = 60000

OUTPUT_FILE = './data/league_data.csv'

# Dane startowe
SEED_NAME = "TryhardEkko"
SEED_TAG = "Ekko"

# Inicjalizacja
lol_watcher = LolWatcher(API_KEY)
riot_watcher = RiotWatcher(API_KEY)

# Zbiory danych
players_to_visit = set()
visited_players = set()
collected_match_ids = set()

ROLE_MAP = {'TOP': 0, 'JUNGLE': 1, 'MIDDLE': 2, 'BOTTOM': 3, 'UTILITY': 4}

Zamienia Nick (Riot ID) na identyfikator PUUID

In [ ]:
def get_puuid_by_riot_id(game_name, tag_line):
    try:
        account = riot_watcher.account.by_riot_id(ROUTING, game_name, tag_line)
        return account['puuid']
    except Exception as e:
        print(f"Błąd PUUID: {e}")
        return None

Funkcja przetwarzająca i filtrująca dane meczowe

In [ ]:
def parse_match_and_collect_players(match_detail):
    info = match_detail['info']

    # Wyciągamy ID (PUUID) wszystkich 10 graczy – potrzebne do "Kuli Śnieżnej",
    # żeby wiedzieć, czyje mecze pobierać jako następne.
    new_puuids = [p['puuid'] for p in info['participants']]

    # FILTR 1: Odrzucamy ARAM-y, tryby eventowe oraz mecze krótsze niż 15 min (900s).
    if info['gameMode'] != 'CLASSIC' or info['gameDuration'] < 900:
        return None, new_puuids

    team_100 = {}
    team_200 = {}

    # Pętla po wszystkich 10 uczestnikach meczu
    for p in info['participants']:
        pos = p.get('teamPosition', '')  # Pobieramy pozycję (TOP, JUNGLE itp.)

        # Jeśli Riot API nie wie, gdzie gracz grał, odrzucamy mecz.
        if pos not in ROLE_MAP: return None, new_puuids

        # Rozdzielamy graczy do drużyn i przypisujemy ich do konkretnych linii (0-4)
        if p['teamId'] == 100:
            # Jeśli na tej linii już ktoś jest (np. dwóch graczy poszło na tą samą linie), odrzucamy mecz
            if ROLE_MAP[pos] in team_100: return None, new_puuids
            team_100[ROLE_MAP[pos]] = p['championId']
        else:
            if ROLE_MAP[pos] in team_200: return None, new_puuids
            team_200[ROLE_MAP[pos]] = p['championId']

    # FILTR 2: Upewniamy się, że każda drużyna ma komplet 5 unikalnych ról (Top, Jgl, Mid, Adc, Supp)
    if len(team_100) != 5 or len(team_200) != 5:
        return None, new_puuids

    row = {}
    # Zapisujemy championów w stałej kolejności (0=Top, 1=Jungle...)
    for r in range(5):
        row[f't1_champ_{r}'] = team_100[r]
    for r in range(5):
        row[f't2_champ_{r}'] = team_200[r]

    # Sprawdzamy wynik
    win_100 = False
    for p in info['participants']:
        if p['teamId'] == 100:
            win_100 = p['win']
            break

    # Zapisujemy wynik
    row['t1_win'] = 1 if win_100 else 0

    return row, new_puuids

Główna pętla algorytmu Kuli Śnieżnej z obsługą limitów API i zapisem przyrostowym

In [ ]:
def main():
    start_puuid = get_puuid_by_riot_id(SEED_NAME, SEED_TAG)
    if not start_puuid:
        print("!!! BŁĄD STARTOWY: Nie znaleziono gracza SEED !!!")
        return

    players_to_visit.add(start_puuid)
    print(f"Start: {SEED_NAME}#{SEED_TAG}")

    matches_data = []
    total_matches_saved = 0

    if os.path.exists(OUTPUT_FILE):
        try:
            df = pd.read_csv(OUTPUT_FILE)
            total_matches_saved = len(df)
            print(f"Wznowiono. Na dysku jest już: {total_matches_saved} wierszy.")
        except:
            pass

    while total_matches_saved < TARGET_MATCHES:
        if not players_to_visit:
            players_to_visit.add(start_puuid)

        current_puuid = players_to_visit.pop()
        if current_puuid in visited_players: continue
        visited_players.add(current_puuid)

        try:
            match_list = lol_watcher.match.matchlist_by_puuid(ROUTING, current_puuid, queue=QUEUE_ID, count=COUNT_MATCHES_PER_PLAYER)
            print(f"Gracz {len(visited_players)}: {len(match_list)} meczów. (Postęp: {total_matches_saved}/{TARGET_MATCHES})")

            for m_id in match_list:
                if m_id in collected_match_ids: continue

                try:
                    match_detail = lol_watcher.match.by_id(ROUTING, m_id)
                    row, new_players = parse_match_and_collect_players(match_detail)

                    for np in new_players:
                        if np not in visited_players: players_to_visit.add(np)

                    if row:
                        matches_data.append(row)
                        collected_match_ids.add(m_id)

                        if len(matches_data) >= 50:
                            df = pd.DataFrame(matches_data)
                            df.to_csv(OUTPUT_FILE, mode='a', header=not os.path.exists(OUTPUT_FILE), index=False)
                            total_matches_saved += len(matches_data)
                            print(f"--- ZAPISANO NA DYSK. RAZEM: {total_matches_saved} ---")
                            matches_data = []

                            if total_matches_saved >= TARGET_MATCHES: return

                except ApiError as e:
                    if e.response.status_code == 429:
                        print("Limit 429 (Mecz). Czekam 10s...")
                        time.sleep(10)
                    elif e.response.status_code == 404: continue
                time.sleep(0.05)

        except ApiError as e:
            if e.response.status_code == 429:
                print("Limit 429 (Gracz). Czekam 60s...")
                time.sleep(60)
                players_to_visit.add(current_puuid)
            elif e.response.status_code == 403:
                print("!!! KLUCZ API WYGASŁ !!! Zmień klucz i zrestartuj.")
                return
        except Exception as e:
            print(f"Błąd: {e}")
            time.sleep(5)

In [ ]:
main()